TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed

## This file differs from 'training_simple_single' by new data generator

In [1]:
input_width = 160
input_height = 100
channels = 3
class_number = 12
data_path = "D:\\Python\\Keras\\Wormax\\data_prepared\\"
model_name = 'models/worm_single_nopooling_tiny_15.h5'
look_forward = 1
use_rotation = False
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers, models, optimizers
import keras.backend as K

def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

def define_model():
    model = models.Sequential()
        
    model.add(layers.Convolution2D(64, (8, 8), strides=(4, 4), activation='relu',
                            input_shape=(input_height, input_width, channels)))
    model.add(layers.Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(layers.Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(class_number, activation='softmax'))    
           
    model.compile(optimizer=optimizers.Adam(lr=1e-3),
                  loss='categorical_crossentropy',
                  metrics=[actual_acc])
    
    model.summary()
    return model

In [3]:
model = define_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 39, 64)        12352     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 18, 64)        65600     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 16, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4719104   
_________________________________________________________________
dense_2 (Dense)              (None, 12)                6156      
Total para

In [4]:
# little prepocessing
from math import atan2, pi
import cv2

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

# Rotating frame and direction counter-clockwise
def get_rotated_frame(img, rotation, target_direction, n_classes):
    if rotation != 0:
        rows, cols = len(img),len(img[0])
        rot_M = cv2.getRotationMatrix2D((cols/2, rows/2), rotation*360/n_classes, 1)
        img = cv2.warpAffine(img, rot_M, (cols, rows))
        target = (target_direction - rotation)%n_classes
    return img, target_direction

In [5]:
import numpy as np
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7

def generator(data_dir, n_classes, role, shuffle=True, batch_size=128):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        arr = np.load(data_dir + listdir[file_i])        
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        # Expanding blocks
        data = []        
        for i in arr:
            for j in i:
                data.append(j)
        data = np.array(data)

        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise 'bad role parameter'
        
        # Generating y
        data_targets = np.zeros((len(data)-look_forward))
        for i in range(len(data_targets)):
            data_targets[i] = get_direction(*data[i+look_forward][1][:2])
            #data_targets[i] = np.array(to_categorical(get_direction(*data[i+look_forward][1][:2]), num_classes=num_classes))
        data = data[:len(data)-look_forward]
        
        # Only X
        data_features = np.zeros((len(data), input_height, input_width, channels))
        for i in range(len(data)):
            data_features[i] = data[i][0][0]
        
        indexes = np.arange(len(data_features)-look_forward)        
        if shuffle:
            np.random.shuffle(indexes)
        
        # Rotating cycle(no rotation for validation)
        rotate_times = n_classes if role == 'train' and use_rotation else 1
        for rot_i in range(rotate_times):
            # Batches forming cycle
            for i in range(0, len(data)-look_forward-batch_size, batch_size):
                samples = data_features.take(indexes[i:i+batch_size], axis=0)
                targets = data_targets.take(indexes[i:i+batch_size], axis=0)
                samples_rot = np.zeros_like(samples)
                targets_rot = np.zeros((batch_size, n_classes))
                
                # Rotating
                for j in range(batch_size):
                    if use_rotation:
                        rotation = (rot_i+j+i)%n_classes
                        img, target = get_rotated_frame(samples[j], rotation, targets[j], n_classes)
                    else:
                        img, target = samples[j], targets[j]
                    samples_rot[j] = img
                    targets_rot[j] = to_categorical(target, num_classes=n_classes)
                    
                # Normilize
                samples_rot = samples_rot / 255
                yield samples_rot, targets_rot

In [6]:
import numpy as np

train_generator = generator(data_path, class_number, 'train', batch_size=16)
validation_generator = generator(data_path, class_number, 'validation', batch_size=32)

print(next(train_generator)[0].shape)
print(next(train_generator)[1].shape)

(16, 100, 160, 3)
(16, 12)


In [7]:
# count class instances count for balancing
if False:
    i = 0
    classes = np.zeros((class_number))
    for samples, targets in generator(data_path, class_number, 'train', batch_size=1024):
        for j in targets:
            classes += j
        i += 1
        if i == 100:
            break
    print(classes)

### tensorboard --logdir=D:\Python\Keras\Wormax\log_dir

# TODO
    2) tune look_forward
    3) change target to angle
    4) fight overfitting on adam
    5) Take away poolingb

I am trying to solve classification task. Basically, teaching game bot on human actions.  
So there are more than 1 right answers for X. But my dataset contains specific actions.

Assume the desirable answer is:`[0, 0.7, 0, 0.7, 0.1]`  
But dataset contains one-hots like: `[0, 0, 0, 1, 0]`

So my idea is to implement Q-state neural network with 2 inputs.  
Q(S,a) - where, S - current state, a - action, result is one number(desire to choose this action).  
And then I can feed to it my X->y dataset.

I am struggling with initializing model. Default


In [ ]:
initial_epoch = 0
if not True:
    model.load_weights(model_name)
    initial_epoch = 176

In [ ]:
if not use_rotation:
    class_weight = {0: 1.15,
                     1: 1.13,
                     2: 1.27,
                     3: 1.51,
                     4: 1.46,
                     5: 1.19,
                     6: 1.0,
                     7: 1.04,
                     8: 1.11,
                     9: 1.2,
                     10: 1.27,
                     11: 1.34}

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir\\' + model_name
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor='val_loss',
        save_best_only=True,
    )
    
]
'''keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.5,                              
        patience=70, 
        min_lr=0.00001
    )'''

steps_per_epoch = 100
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=500,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True,
                            class_weight=class_weight,
                            callbacks=callbacks,
                            initial_epoch=initial_epoch
                        )

Epoch 1/500
100/100 [==============================] - 6s 59ms/step - loss: 2.8783 - actual_acc: 0.1756 - val_loss: 2.3297 - val_actual_acc: 0.2158
Epoch 2/500
100/100 [==============================] - 3s 26ms/step - loss: 2.4154 - actual_acc: 0.3513 - val_loss: 2.4203 - val_actual_acc: 0.2224
Epoch 3/500
100/100 [==============================] - 5s 53ms/step - loss: 2.1039 - actual_acc: 0.4394 - val_loss: 2.6425 - val_actual_acc: 0.2100
Epoch 4/500
100/100 [==============================] - 20s 196ms/step - loss: 2.0746 - actual_acc: 0.4181 - val_loss: 2.2639 - val_actual_acc: 0.1984
Epoch 5/500
100/100 [==============================] - 3s 27ms/step - loss: 2.2961 - actual_acc: 0.3438 - val_loss: 2.2475 - val_actual_acc: 0.2217
Epoch 6/500
100/100 [==============================] - 11s 108ms/step - loss: 1.8397 - actual_acc: 0.4875 - val_loss: 2.3716 - val_actual_acc: 0.2006
Epoch 7/500
100/100 [==============================] - 3s 27ms/step - loss: 1.4783 - actual_acc: 0.6006 - va

Epoch 56/500
100/100 [==============================] - 9s 91ms/step - loss: 2.0339 - actual_acc: 0.4281 - val_loss: 1.9522 - val_actual_acc: 0.3263
Epoch 57/500
100/100 [==============================] - 3s 27ms/step - loss: 1.6452 - actual_acc: 0.5406 - val_loss: 1.9450 - val_actual_acc: 0.3503
Epoch 58/500
100/100 [==============================] - 11s 110ms/step - loss: 1.4140 - actual_acc: 0.6050 - val_loss: 2.1081 - val_actual_acc: 0.3081
Epoch 59/500
100/100 [==============================] - 11s 110ms/step - loss: 1.5147 - actual_acc: 0.5706 - val_loss: 2.0394 - val_actual_acc: 0.3074
Epoch 60/500
100/100 [==============================] - 9s 93ms/step - loss: 2.0375 - actual_acc: 0.4106 - val_loss: 1.9629 - val_actual_acc: 0.3343
Epoch 61/500
100/100 [==============================] - 3s 28ms/step - loss: 1.7356 - actual_acc: 0.5094 - val_loss: 1.9812 - val_actual_acc: 0.3656
Epoch 62/500
100/100 [==============================] - 9s 87ms/step - loss: 1.5304 - actual_acc: 0.54

Q:\Program Files\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.155589). Check your callbacks.
  % delta_t_median)
Q:\Program Files\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.135139). Check your callbacks.
  % delta_t_median)


  5/100 [>.............................] - ETA: 11s - loss: 2.2400 - actual_acc: 0.3000

Q:\Program Files\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.114690). Check your callbacks.
  % delta_t_median)


100/100 [==============================] - 7s 74ms/step - loss: 1.8929 - actual_acc: 0.4387 - val_loss: 1.7891 - val_actual_acc: 0.3452
Epoch 100/500
100/100 [==============================] - 8s 77ms/step - loss: 1.6599 - actual_acc: 0.5056 - val_loss: 1.9636 - val_actual_acc: 0.3161
Epoch 101/500
100/100 [==============================] - 11s 105ms/step - loss: 1.9918 - actual_acc: 0.4256 - val_loss: 1.8291 - val_actual_acc: 0.3895
Epoch 102/500
100/100 [==============================] - 10s 105ms/step - loss: 1.8179 - actual_acc: 0.4806 - val_loss: 1.8795 - val_actual_acc: 0.3350
Epoch 103/500
100/100 [==============================] - 3s 28ms/step - loss: 1.5053 - actual_acc: 0.5856 - val_loss: 2.0633 - val_actual_acc: 0.3299
Epoch 104/500
100/100 [==============================] - 10s 97ms/step - loss: 1.3158 - actual_acc: 0.6162 - val_loss: 1.9028 - val_actual_acc: 0.3706
Epoch 105/500
100/100 [==============================] - 11s 115ms/step - loss: 2.0154 - actual_acc: 0.4169 -

100/100 [==============================] - 3s 27ms/step - loss: 1.7991 - actual_acc: 0.4706 - val_loss: 1.9168 - val_actual_acc: 0.3161
Epoch 154/500
100/100 [==============================] - 11s 107ms/step - loss: 1.5417 - actual_acc: 0.5569 - val_loss: 1.9479 - val_actual_acc: 0.3110
Epoch 155/500
100/100 [==============================] - 12s 115ms/step - loss: 1.7921 - actual_acc: 0.4694 - val_loss: 1.8121 - val_actual_acc: 0.3503
Epoch 156/500
100/100 [==============================] - 10s 103ms/step - loss: 2.0900 - actual_acc: 0.3769 - val_loss: 1.8419 - val_actual_acc: 0.3328
Epoch 157/500
100/100 [==============================] - 3s 27ms/step - loss: 1.8053 - actual_acc: 0.4650 - val_loss: 1.8224 - val_actual_acc: 0.3307
Epoch 158/500
100/100 [==============================] - 11s 106ms/step - loss: 1.6792 - actual_acc: 0.5012 - val_loss: 1.9416 - val_actual_acc: 0.3227
Epoch 159/500
100/100 [==============================] - 12s 122ms/step - loss: 1.7601 - actual_acc: 0.488

In [ ]:
from keras import backend as K
K.clear_session()

# Урок
    1) автоматизируйте подбор гиперпараметров
    2) baseline